In [ ]:
!pip install datasets
!pip install transformers
!pip install nltk
!git clone https://github.com/Yale-LILY/QMSum.git
!pip install sacrebleu
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 92.7 MB/s eta 0:00

In [ ]:
import json

# read the dataset
# please enter the path of your data
split = 'train'
data_path = 'QMSum/data/ALL/jsonl/' + split + '.jsonl'
data = []
with open(data_path) as f:
    for line in f:
        data.append(json.loads(line))
n_meetings = len(data)
print('Total {} meetings in the {} set.'.format(n_meetings, split))

Total 162 meetings in the train set.


In [ ]:
data[0]

{'topic_list': [{'topic': 'Remote control prototype introduction',
   'relevant_text_span': [['0', '53']]},
  {'topic': 'Product evaluation discussion',
   'relevant_text_span': [['54', '283']]},
  {'topic': 'Product cost discussion', 'relevant_text_span': [['284', '459']]},
  {'topic': 'Project and overall process review',
   'relevant_text_span': [['460', '505']]}],
 'general_query_list': [{'query': 'Summarize the whole meeting.',
   'answer': "Firstly, Project Manager reviewed the overall process that the team designed the remote control that they had viewed the fashion trends and incorporated the key features. After that, User Interface introduced the prototype of remote control based on the previous discussion of its function. The prototype was yellow like a banana with a simplest quick on-off button and could be used both-handed. Secondly, Marketing designed an evaluation test according to criteria found of the market trend as well as the customers' complaints and the team gave o

In [ ]:
dump_data = []
for i,item in enumerate(data[0]['meeting_transcripts']):
  dump_data.append({'speaker':item['speaker'],'content':item['content']})

In [ ]:
json_object = json.dumps(dump_data, indent=4)

with open("1_meet.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
from nltk import word_tokenize
# tokneize a sent
def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

In [ ]:
# filter some noises caused by speech recognition
def clean_data(text):
    text = text.replace('{ vocalsound } ', '')
    text = text.replace('{ disfmarker } ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause } ', '')
    text = text.replace('{ nonvocalsound } ', '')
    text = text.replace('{ gap } ', '')
    return text

In [ ]:
import nltk
nltk.download('punkt')

# process data for BART
# the input of the model here is the gold span corresponding to each query
bart_data_gold = []
for i in range(len(data)):
    # get meeting content
    entire_src = []
    for k in range(len(data[i]['meeting_transcripts'])):
        cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
        cur_turn = cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])
        entire_src.append(cur_turn)
    entire_src = ' '.join(entire_src)
    for j in range(len(data[i]['general_query_list'])):
        cur = {}
        query = tokenize(data[i]['general_query_list'][j]['query'])
        cur['src'] = clean_data('<s> ' + query + ' </s> ' + entire_src + ' </s>')
        target = tokenize(data[i]['general_query_list'][j]['answer'])
        cur['tgt'] = target
        bart_data_gold.append(cur)
    # for j in range(len(data[i]['specific_query_list'])):
    #     cur = {}
    #     query = tokenize(data[i]['specific_query_list'][j]['query'])
    #     src = []
    #     # get the content in the gold span for each query
    #     for span in data[i]['specific_query_list'][j]['relevant_text_span']:
    #         assert len(span) == 2
    #         st, ed = int(span[0]), int(span[1])
    #         for k in range(st, ed + 1):
    #             cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
    #             cur_turn = cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])
    #             src.append(cur_turn)
    #     src = ' '.join(src)
    #     cur['src'] = clean_data('<s> ' + query + ' </s> ' + src + ' </s>')
    #     target = tokenize(data[i]['specific_query_list'][j]['answer'])
    #     cur['tgt'] = target
    #     bart_data_gold.append(cur)

print('Total {} query-summary pairs in the {} set'.format(len(bart_data_gold), split))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer,AutoConfig,LEDConfig
from transformers import AutoModelForSeq2SeqLM,LEDForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("pszemraj/led-large-book-summary")
model = AutoModelForSeq2SeqLM.from_pretrained("pszemraj/led-large-book-summary")

In [ ]:
from datasets import DatasetDict
from datasets import Dataset

dataset_dict = DatasetDict({
    "train": Dataset.from_dict(dict(zip(bart_data_gold[0].keys(), zip(*[d.values() for d in bart_data_gold]))))
})

In [ ]:
split_datasets = dataset_dict["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples['src'], truncation=True)
    labels = tokenizer(examples['tgt'], truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset_dict["train"].column_names,
)

Map:   0%|          | 0/145 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets.push_to_hub(repo_id='tokenized_datasets', private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import DatasetDict
from datasets import Dataset
from datasets import load_dataset
tokenized_datasets = load_dataset('fgiuhsdfkjhfv/tokenized_datasets', use_auth_token=True)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/145 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17 [00:00<?, ? examples/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"longsec_general_query",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    tf32=True,
    gradient_checkpointing=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    optim="adafactor",
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,)

Cloning https://huggingface.co/fgiuhsdfkjhfv/longsec_general_query into local empty directory.


In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
trainer.push_to_hub(tags="text2text", commit_message="Training complete") # อัพขึ้น hugging face